<a href="https://colab.research.google.com/github/oscar0830/AIAgentClass/blob/dev/Prompt_Engineering_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Customer Service AI (console version)
- Warm welcome + clarification on first interaction
- Intent classification (simple rule-based)
- FAQ knowledge base responses
- Escalation to human when confidence is low / unknown
"""

from dataclasses import dataclass, asdict
from datetime import datetime
import json
import re
import uuid


# ----------------------------
# Data models
# ----------------------------

@dataclass
class EscalationTicket:
    ticket_id: str
    created_at: str
    customer_name: str
    customer_contact: str
    issue_summary: str
    conversation_snippet: str
    priority: str = "normal"


# ----------------------------
# Knowledge base (edit for your business)
# ----------------------------

FAQ = [
    {
        "intent": "hours",
        "patterns": [
            r"\bhours\b", r"\bopen\b", r"\bclosing\b", r"\bclose\b", r"\bwhen are you open\b"
        ],
        "answer": "We’re open Monday–Friday, 9 AM–6 PM (local time). Weekends: 10 AM–4 PM."
    },
    {
        "intent": "refund",
        "patterns": [
            r"\brefund\b", r"\breturn\b", r"\bmoney back\b", r"\bexchange\b"
        ],
        "answer": "Refunds are available within 30 days of purchase with a receipt. Returns are processed in 3–5 business days."
    },
    {
        "intent": "shipping",
        "patterns": [
            r"\bshipping\b", r"\bdelivery\b", r"\bship\b", r"\btracking\b", r"\border status\b"
        ],
        "answer": "Standard shipping is 3–7 business days. If you share your order number, I can help you interpret tracking updates."
    },
    {
        "intent": "pricing",
        "patterns": [
            r"\bprice\b", r"\bcost\b", r"\bpricing\b", r"\bquote\b"
        ],
        "answer": "Pricing depends on the product/service. Tell me which item or plan you’re looking at and any quantity/requirements."
    },
    {
        "intent": "account_login",
        "patterns": [
            r"\blogin\b", r"\bsign in\b", r"\bpassword\b", r"\breset\b", r"\baccount\b"
        ],
        "answer": "If you can’t log in, try a password reset. If you tell me whether you’re seeing an error message, I can guide you."
    }
]


# ----------------------------
# Core AI logic (rule-based)
# ----------------------------

def detect_intent(message: str):
    """
    Returns (intent, score, matched_pattern).
    Score is a simple heuristic: number of pattern matches.
    """
    msg = message.lower()
    best = ("unknown", 0, None)

    for item in FAQ:
        score = 0
        matched = None
        for pat in item["patterns"]:
            if re.search(pat, msg):
                score += 1
                matched = pat
        if score > best[1]:
            best = (item["intent"], score, matched)

    return best


def answer_for_intent(intent: str):
    for item in FAQ:
        if item["intent"] == intent:
            return item["answer"]
    return None


def needs_escalation(intent: str, score: int, message: str) -> bool:
    """
    Escalate if unknown or confidence is low, or user signals urgency/anger.
    """
    msg = message.lower()
    urgent_signals = ["complaint", "angry", "fraud", "chargeback", "lawsuit", "legal", "cancel", "manager", "supervisor"]
    very_negative = ["this is terrible", "worst", "unacceptable", "fix now", "furious"]

    if intent == "unknown" or score == 0:
        return True
    if any(w in msg for w in urgent_signals) or any(w in msg for w in very_negative):
        return True

    # Low-confidence: only 1 weak match AND message is long/complex
    if score == 1 and len(msg.split()) > 18:
        return True

    return False


# ----------------------------
# Escalation handling
# ----------------------------

def create_ticket(customer_name: str, customer_contact: str, issue_summary: str, convo_snippet: str) -> EscalationTicket:
    ticket = EscalationTicket(
        ticket_id=str(uuid.uuid4())[:8].upper(),
        created_at=datetime.now().isoformat(timespec="seconds"),
        customer_name=customer_name,
        customer_contact=customer_contact,
        issue_summary=issue_summary.strip()[:200],
        conversation_snippet=convo_snippet.strip()[:400],
        priority="normal"
    )
    return ticket


def save_ticket(ticket: EscalationTicket, path: str = "escalation_tickets.jsonl"):
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(asdict(ticket)) + "\n")


# ----------------------------
# Conversation flow
# ----------------------------

def first_interaction():
    print("🤝 Hello! Welcome to Acme Customer Support.")
    print("I’m here to help, and I’ll ask a couple quick questions so I can get you the best answer.\n")

    name = input("1) What’s your name? ").strip() or "Customer"
    contact = input("2) What’s the best email or phone number to reach you (in case we need to escalate)? ").strip() or "N/A"
    issue = input("3) Briefly describe what you need help with today: ").strip()

    # Clarification questions (first interaction requirement)
    print("\nThanks! Two quick clarifying questions:")
    order = input("- Do you have an order number (if relevant)? If yes, type it; if not, type 'no': ").strip()
    urgency = input("- Is this urgent (e.g., service down, payment issue)? yes/no: ").strip().lower()

    context = {
        "name": name,
        "contact": contact,
        "issue": issue,
        "order_number": order,
        "urgent": urgency.startswith("y"),
    }
    return context


def run_chat():
    context = first_interaction()
    conversation_log = []

    print("\n✅ Got it. Tell me a bit more, or ask your question directly. Type 'quit' to exit.\n")

    while True:
        user_msg = input(f"{context['name']}: ").strip()
        if user_msg.lower() in ("quit", "exit"):
            print("Support Bot: Thanks for contacting us. Have a great day!")
            break

        conversation_log.append(f"{context['name']}: {user_msg}")

        intent, score, _ = detect_intent(user_msg)
        if needs_escalation(intent, score, user_msg):
            # Create ticket and escalate
            issue_summary = context["issue"] or user_msg
            snippet = "\n".join(conversation_log[-6:])  # last few messages
            ticket = create_ticket(context["name"], context["contact"], issue_summary, snippet)
            save_ticket(ticket)

            print("\nSupport Bot: I want to make sure this gets handled correctly.")
            print("Support Bot: I’m escalating this to a human support specialist now.")
            print(f"Support Bot: Your ticket ID is {ticket.ticket_id}.")
            print("Support Bot: A team member will reach out using the contact info you provided.\n")

            # Optional: end chat after escalation
            continue

        # Provide answer + a follow-up question for continued clarity
        answer = answer_for_intent(intent)
        print(f"\nSupport Bot: {answer}")
        print("Support Bot: Does that solve it, or is there a specific detail (order #, error message, product name) you can share?\n")


if __name__ == "__main__":
    run_chat()

🤝 Hello! Welcome to Acme Customer Support.
I’m here to help, and I’ll ask a couple quick questions so I can get you the best answer.

1) What’s your name? oo
